In [ ]:
# import statements
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
# from keras.backend import image_data_format
from tensorflow.keras import mixed_precision
from tensorflow.keras.activations import relu
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
mixed_precision.set_global_policy('mixed_float16')
# using mixed_precision speeds up training on GPUs with compute-compatibility >= 8.0
# it also halves the memory usage as keras intelligently switched between 16-bit and 32-bit precision

In [ ]:
# some fixed parameters
# this code assumes that the dataset is in the same directory as the script
# it also assumes that the best model saved file is in the same directory with the name "best_model.h5"
IMG_SIZE = (300, 300)
BASE_PATH = "./inaturalist_12K"
names = ["Amphibia", "Animalia", "Arachnida", "Aves", "Fungi", "Insecta", "Mammalia", "Mollusca", "Plantae", "Reptilia"]

In [ ]:
### BEST PARAMETERS ###
NUM_FILTERS = 32
CONV_FILTER_SIZE = 5
FILTER_MULT = 2
DATA_AUGMENTATION = False
DROPOUT = 0.2
BATCHNORM = False
OPTIM_NAME = "adamax"
LR = 7e-5
BATCH_SIZE = 32
DENSE_UNITS = 256
DENSE_ACTIVATION = "relu"

In [ ]:
# a function for reading data from the given directory structure
# we rescale our image pixel values by 1/255 but this time we use the whole training data to get the train-test metrics
# returns the train and test generators which can be passed to the model.evaluate() method
def get_data_generators(data_augmentation=True, img_size=IMG_SIZE, batch_size=32):
    if data_augmentation:
        # the following augmentation techniques are used
        train_data = ImageDataGenerator(rescale=1/255,
                                        samplewise_center=True,
                                        samplewise_std_normalization=True,
                                        shear_range=0.25,
                                        zoom_range=[0.25, 1.25],
                                        width_shift_range=0.25,
                                        height_shift_range=0.25,
                                        horizontal_flip=True,
                                        rotation_range=60)
    else:
        train_data = ImageDataGenerator(rescale=1/255)

    train_gen = train_data.flow_from_directory(f"{BASE_PATH}/train",
                                         target_size=img_size,
                                         batch_size=batch_size,
                                         color_mode="rgb",
                                         class_mode="sparse",
                                         shuffle=True,
                                         seed=123,
                                         subset="training")
    # this time read the test-data as well
    # but for evaluation we do not do any augmentation on the test data                       
    test_data = ImageDataGenerator(rescale=1/255)
    test_gen = test_data.flow_from_directory(f"{BASE_PATH}/val",
                                             target_size=img_size,
                                             batch_size=batch_size,
                                             color_mode="rgb",
                                             batch_size=1,
                                             seed=123,
                                             class_mode="sparse",
                                             shuffle=True)

    return train_gen, test_gen

# 4a) Evaluate the Model

In [ ]:
# get the train, validation and test data
train_gen, test_gen = get_data_generators(data_augmentation=DATA_AUGMENTATION, 
                                          batch_size=BATCH_SIZE)    
# load the saved model       
model = load_model("best_model.h5")
# evaluate the model
model.evaluate(train_gen)
model.evaluate(test_gen)
# visualize the layers of the model
model.summary()

# 4b) Visualize Predictions of the Model
### Disclaimer: From now on, images in report might differ from the ones when you run the code, since random images are chosen during every run

In [ ]:
# collect 30 images, 3 for each class
images, c = [[], [], [], [], [], [], [], [], [], []], 0
while c < 30:
    img, label = test_gen.next()
    label = int(label)
    if len(images[label]) < 3:
        images[label].append(img)
        c += 1

# plot the images as a 10x3 grid with title and ylabel
plt.figure(figsize=(20, 60))
images = np.squeeze(np.array(images), axis=2)
for i in range(30):
    ax = plt.subplot(10, 3, i+1)
    x = images[i//3][i % 3]
    # predict the class of the input image
    y = np.argmax(model.predict(np.array([x])))
    # set the ylabel as the true class label in blue
    plt.ylabel(names[i//3], color="blue", fontdict={'fontsize': 15, 'fontweight': 'bold'})
    color = "red" if names[i//3] != names[y] else "green"
    # set the title as the prediction by the model
    # the title is red if the prediction is incorrect else green
    plt.title(names[y], color=color, fontdict={'fontsize': 15, 'fontweight': 'bold'})
    plt.imshow(x)
# save the plot and manually upload to wandb
plt.savefig("prediction_table", dpi=300, bbox_inches="tight", pad_inches=0)

# 4c) Visualize Filters and Feature-Maps of First Layer

In [ ]:
# choose a random image for plotting filters and feature-maps
# plot the original image
image, label = test_gen.next()
image, label = np.squeeze(image, axis=0), int(label)
temp_model = Model(inputs=model.inputs, outputs=model.layers[0].output)
plt.title(f"{names[label]}", fontdict={'fontsize': 15, 'fontweight': 'bold'})
plt.imshow(image)
plt.axis("off")
plt.show()
plt.savefig("feature_maps_filters_original", dpi=360, bbox_inches="tight", pad_inches=0)

# get the filters
filters, _ = model.layers[0].get_weights()
n_filters = filters.shape[-1]
cols, rows = 8, n_filters//8
# scale the filter values
filters = (filters - filters.min()) / (filters.max() - filters.min())

# plot the filters in a grid
plt.figure(figsize=(30, 10))
for i in range(n_filters):
    ax = plt.subplot(rows, cols, i+1)
    plt.imshow(filters[:, :, :, i])
    plt.title(f"visualization of filter_{i+1}", fontdict={'fontsize': 10, 'fontweight': 'bold'})
    plt.axis("off")
plt.savefig("filters_visualization", dpi=360, bbox_inches="tight", pad_inches=0)

# get the feature maps as the output of the first layer
# plot the feature maps as a grid
feature_maps = temp_model.predict(np.array([image]))
plt.figure(figsize=(30, 10))
for i in range(n_filters):
    ax = plt.subplot(rows, cols, i+1)
    plt.imshow(feature_maps[0,:,:,i])
    plt.title(f"visualization of feature_map_{i+1}", fontdict={'fontsize': 10, 'fontweight': 'bold'})
    plt.axis("off")
plt.savefig("feature_map_visualization", dpi=360, bbox_inches="tight", pad_inches=0)

# 5) Guided Backpropogation

In [ ]:
# required functions for guided backprop
# define a custom derivative of relu as discussed in class
@tf.custom_gradient
def guidedRelu(input):
    def grad(upstream):
        return tf.cast(upstream > 0, "float32") * tf.cast(input > 0, "float32") * upstream
    return tf.nn.relu(input), grad

# a function convert the gradient values into a proper image
# for this, standardize the values, clip them between [0, 1] and multiply by 255 to get proper pixel values
def deprocess_image(x):
    x = (x - x.mean())/(x.std() + 1e-8)
    x *= 0.25
    x += 0.5
    x = np.clip(x, 0, 1)
    x *= 255

    # if image_data_format() == "channels_first":
    #     x = x.transpose((1, 2, 0))
    return x.astype("uint8")

In [ ]:
# create a guided backprop model
guided_backprop_model = Model(inputs = [model.inputs], outputs = [model.get_layer("conv2d_4").output])
# find the layers which have activation
layer_dict = [layer for layer in guided_backprop_model.layers[1:] if hasattr(layer, "activation")]
out_shape = model.get_layer("conv2d_4").output.shape[1:]

# choose a random image and plot it
image, label = test_gen.next()
image, label = np.squeeze(image, axis=0), int(label)
plt.title(f"{names[label]}", fontdict={'fontsize': 15, 'fontweight': 'bold'})
plt.imshow(image)
plt.axis("off")
plt.show()
plt.savefig("guided_backprop_original", dpi=360, bbox_inches="tight", pad_inches=0)

# if the activation in the layers we found was relu
# change it to guidedRelu to guide the gradients to specific locations
for layer in layer_dict:
    if layer.activation == relu:
        layer.activation = guidedRelu

imgs = []
for i in range(10):
    # choose a random neuron from fliters and apply guided backprop to it
    # like this, do it for 10 random neurons
    neuron = [np.random.randint(0, j-1) for j in out_shape]
    mask = np.zeros(out_shape)
    mask[neuron[0], neuron[1], neuron[2]] = 1
    # keep track of the gradients with GradientTape
    with tf.GradientTape() as tape:
        inputs = tf.cast([image], tf.float32)
        tape.watch(inputs)
        outputs = guided_backprop_model(inputs) * mask
    # at the end, convert them from tensors to numpy arrays for plotting
    gradients = tape.gradient(outputs, inputs).numpy()[0]
    # keep track of which neuron produced what gradients
    imgs.append((neuron, gradients))

# plot the guided backprop images for those 10 random neurons
plt.figure(figsize=(60, 60))
for i, (neuron, gradients) in enumerate(imgs):
    ax = plt.subplot(10, 1, i+1)
    plt.imshow(deprocess_image(gradients))
    plt.title(f"guided_backprop for neuron-{neuron}", fontdict={'fontsize': 12, 'fontweight': 'bold'})
    plt.axis("off")
plt.savefig("guided_backprop_visualization", dpi=360, bbox_inches="tight", pad_inches=0)